In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 50

# Reading CSV

In [ ]:
# dtypes = {'term':'str', 'grade':'str', 'sub_grade':'str', 
#           'emp_title':'str', 'emp_length':'str', 'home_ownership':'str',
#           'verification_status':'str', 'issue_d': 'str', 'loan_status':'str',
#           'purpose':'str', 'add_state':'str', 
#           'sec_app_earliest_cr_line':'str', 'earliest_cr_line':'str',
#           'initial_list_status':'str', 'last_pymnt_d':'str', 
#           'last_credit_pull_d':'str'}

# accepted = pd.read_csv('/Users/ivanpassoni/Downloads/lending-club/accepted_2007_to_2018Q4.csv',
#                        low_memory = False, dtype = dtypes)

# Dropping bad Id's

In [ ]:
# remove bad id's
# accepted.drop(index= accepted.loc[accepted['id'].apply(lambda s: len(str(s))) > 10, :].index, inplace = True)

# Parsing Datas

In [ ]:
# # Parsing issue_date, last_pymnt_d, earliest_cr_line, last_credit_pull_d datetime
# def parse_date(d):
#     return pd.to_datetime(d, format='%b-%Y')

# accepted['issue_d']            = accepted['issue_d'].apply(parse_date)
# accepted['last_pymnt_d']       = accepted['last_pymnt_d'].apply(parse_date)
# accepted['earliest_cr_line']   = accepted['earliest_cr_line'].apply(parse_date)
# accepted['last_credit_pull_d'] = accepted['last_credit_pull_d'].apply(parse_date)
# next_pymnt_d will be droped later

# Saved and reading the data parsed

In [ ]:
# accepted.to_csv('/Users/ivanpassoni/Downloads/lending-club/accepted_2007_to_2018Q4_parsed_data.csv', index = False)

In [4]:
# dtypes = {'term':'str', 'grade':'str', 'sub_grade':'str', 
#           'emp_title':'str', 'emp_length':'str', 'home_ownership':'str',
#           'verification_status':'str', 'issue_d': 'str', 'loan_status':'str',
#           'purpose':'str', 'add_state':'str', 
#           'sec_app_earliest_cr_line':'str', 'earliest_cr_line':'str',
#           'initial_list_status':'str', 'last_pymnt_d':'str', 
#           'last_credit_pull_d':'str'}

parse_dates = ['issue_d', 'last_pymnt_d', 'earliest_cr_line', 'last_credit_pull_d']

accepted = pd.read_csv('/Users/ivanpassoni/Downloads/lending-club/accepted_2007_to_2018Q4_parsed_data.csv',
                       low_memory = False, dtype = dtypes, parse_dates = parse_dates)

# Merging joint and individual loan applications

We consider a joint application as an individual application with the annual_inc, dti and revol_bal given by their joint values.

In [5]:
# Considering annual_inc == annual_inc_joint, dti == dti_joint
accepted['annual_inc'] = accepted[['application_type', 'annual_inc', 'annual_inc_joint']]\
.apply(lambda row: row['annual_inc'] if row['application_type'] == 'Individual' else row['annual_inc_joint'], axis = 1)

accepted['dti'] = accepted[['application_type', 'dti', 'dti_joint']]\
.apply(lambda row: row['dti'] if row['application_type'] == 'Individual' else row['dti_joint'], axis = 1)

accepted['revol_bal'] = accepted[['application_type', 'revol_bal', 'revol_bal_joint']]\
.apply(lambda row: row['revol_bal'] if row['application_type'] == 'Individual' else row['revol_bal_joint'], axis = 1)

# Imputing Last Payment Date

We impute the last_pymnt_d by adding to issue_d the number of months given by total_pymnt/installment

In [6]:
from dateutil.relativedelta import relativedelta
from datetime import timedelta

def increment_months(s, length):
    while length > 0:
        if s.month == 12:
            s = s + timedelta(days = 31)
        else:
            s = s + relativedelta(months = 1)
        length -= 1
    return s

accepted['last_pymnt_d'].fillna(accepted.loc[accepted['last_pymnt_d'].isnull(), 
                                             ['issue_d', 'total_pymnt', 'installment']]\
                        .apply(lambda s: increment_months(s[0],np.floor(s[1]/s[2])), axis = 1),
                                inplace = True)

# Defining the Apriori columns and droping irrelevant columns

In [7]:
apriori_columns = ['id', 'member_id', 'loan_amnt', 'funded_amnt', 
                   'funded_amnt_inv','term','int_rate',
                   'installment','grade','sub_grade','emp_title',
                   'emp_length','home_ownership', 'annual_inc',
                   'issue_d','url','desc','purpose','title',
                   'zip_code','addr_state','dti','delinq_2yrs',
                   'earliest_cr_line','fico_range_low',
                   'fico_range_high','inq_last_6mths',
                   'mths_since_last_delinq','mths_since_last_record',
                   'open_acc','pub_rec','revol_bal','revol_util',
                   'total_acc','initial_list_status',
                   'collections_12_mths_ex_med',
                   'mths_since_last_major_derog','application_type',
                   'annual_inc_joint','dti_joint','acc_now_delinq',
                   'tot_coll_amt','tot_cur_bal','open_acc_6m',
                   'open_act_il','open_il_12m','open_il_24m',
                   'mths_since_rcnt_il','total_bal_il','il_util',
                   'open_rv_12m','open_rv_24m','max_bal_bc','all_util',
                   'total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m',
                   'acc_open_past_24mths','avg_cur_bal','bc_open_to_buy',
                   'bc_util','chargeoff_within_12_mths','delinq_amnt',
                   'mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op',
                   'mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc',
                   'mths_since_recent_revol_delinq','num_accts_ever_120_pd',
                   'num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl',
                   'num_il_tl','num_op_rev_tl','num_rev_accts',
                   'num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m',
                   'num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m',
                   'pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies',
                   'tax_liens','tot_hi_cred_lim','total_bal_ex_mort',
                   'total_bc_limit','total_il_high_credit_limit',
                   'revol_bal_joint','sec_app_fico_range_low',
                   'sec_app_fico_range_high','sec_app_earliest_cr_line',
                   'sec_app_inq_last_6mths','sec_app_mort_acc',
                   'sec_app_open_acc','sec_app_revol_util',
                   'sec_app_open_act_il','sec_app_num_rev_accts',
                   'sec_app_chargeoff_within_12_mths',
                   'sec_app_collections_12_mths_ex_med',
                   'sec_app_mths_since_last_major_derog', 
                   'disbursement_method']

## Droping some columns

In [8]:
# Drop irrelevant columns
hardships = [col for col in accepted.columns if 'hard' in col]
hardships.extend(['payment_plan_start_date', 'orig_projected_additional_accrued_interest'])

investors = [col for col in accepted.columns if 'inv' in col]

secundary_applicants = [col for col in accepted.columns if 'sec' in col]

settlements = [col for col in accepted.columns if 'settlement' in col]

joint = [col for col in accepted.columns if 'joint' in col]

drop_columns = ['member_id', 'url', 'desc', 
                'pymnt_plan', 'title', 
                'zip_code', 'disbursement_method',
                'next_pymnt_d', 'policy_code',
                'deferral_term', 'emp_title',
                'application_type']     +\
                hardships + settlements           +\
                secundary_applicants + investors  +\
                joint

accepted.drop(columns = drop_columns, inplace = True)

# updating the apriori columns list

apriori_columns = [col for col in apriori_columns if col not in drop_columns]

# From now on, we consider only loans before 2015

In [9]:
accepted2 = accepted.loc[accepted['issue_d'] < '2015-01-1', :]

# Filling the apriori columns NA's

Finding missing values below .01 percent

In [10]:
null_sum = accepted2[apriori_columns].isnull().sum()
p_null_sum = null_sum[null_sum > 0]
p_null_sum[p_null_sum.sort_values(ascending = False)/accepted2.shape[0] < .01].sort_values(ascending = False)

pub_rec_bankruptcies          1365
revol_util                     340
chargeoff_within_12_mths       145
collections_12_mths_ex_med     145
tax_liens                      105
delinq_amnt                     29
acc_now_delinq                  29
total_acc                       29
pub_rec                         29
open_acc                        29
inq_last_6mths                  29
earliest_cr_line                29
delinq_2yrs                     29
annual_inc                       4
dtype: int64

Fill NA's of pub_rec_bankruptcies and drop observations that have missing values of the rest of the columns

In [11]:
accepted2['pub_rec_bankruptcies'].fillna(0, inplace = True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [12]:
columns_na = p_null_sum[p_null_sum.sort_values(ascending = False)/accepted2.shape[0] < .01].index

In [13]:
idx = accepted2.loc[accepted2[columns_na].isnull().any(axis = 1), :].index
accepted2 = accepted2.drop(index = idx)

### Updating the NA list

In [14]:
null_sum = accepted2[apriori_columns].isnull().sum()
p_null_sum = null_sum[null_sum > 0]
p_null_sum.sort_values(ascending = False)/accepted2.shape[0]

open_il_24m                       1.000000
open_il_12m                       1.000000
total_cu_tl                       1.000000
inq_fi                            1.000000
all_util                          1.000000
max_bal_bc                        1.000000
open_rv_24m                       1.000000
open_rv_12m                       1.000000
il_util                           1.000000
total_bal_il                      1.000000
mths_since_rcnt_il                1.000000
inq_last_12m                      1.000000
open_act_il                       1.000000
open_acc_6m                       1.000000
mths_since_last_record            0.865780
mths_since_last_major_derog       0.787687
mths_since_recent_revol_delinq    0.701444
mths_since_last_delinq            0.537026
num_tl_120dpd_2m                  0.167760
pct_tl_nvr_dlq                    0.150668
avg_cur_bal                       0.150340
tot_cur_bal                       0.150340
tot_coll_amt                      0.150340
total_rev_h

We drop all the columns that have more than 50% missing values. In this case, the only one that is close to 50% is the mths_since_last_delinq

In [15]:
drop_na_columns = p_null_sum[p_null_sum.sort_values(ascending = False)/accepted2.shape[0] > .5].index

In [16]:
accepted2.drop(columns = drop_na_columns, inplace = True)

# We also update the apriori_columns
apriori_columns = [col for col in apriori_columns if col not in drop_na_columns]

### Updating the NA list again

In [17]:
null_sum = accepted2[apriori_columns].isnull().sum()
p_null_sum = null_sum[null_sum > 0]
p_null_sum.sort_values(ascending = False)

num_tl_120dpd_2m              78158
pct_tl_nvr_dlq                70195
total_il_high_credit_limit    70042
num_rev_accts                 70042
mo_sin_old_rev_tl_op          70042
num_accts_ever_120_pd         70042
num_actv_bc_tl                70042
num_actv_rev_tl               70042
avg_cur_bal                   70042
num_bc_tl                     70042
num_il_tl                     70042
num_op_rev_tl                 70042
num_rev_tl_bal_gt_0           70042
mo_sin_rcnt_tl                70042
total_rev_hi_lim              70042
num_tl_30dpd                  70042
num_tl_90g_dpd_24m            70042
num_tl_op_past_12m            70042
tot_cur_bal                   70042
tot_hi_cred_lim               70042
tot_coll_amt                  70042
mo_sin_rcnt_rev_tl_op         70042
num_bc_sats                   58365
num_sats                      58365
bc_util                       53805
percent_bc_gt_75              53711
bc_open_to_buy                53596
mths_since_recent_bc        

### Fill emp_length NA's

In [18]:
a = accepted2['emp_length'].value_counts()/accepted2['emp_length'].shape[0]*1000
a = a.apply(lambda x: int(x))
x = pd.Series(range(1, np.sum(a)+1))

n = np.sum(accepted2['emp_length'].isnull())

x = x.sample(n, replace = True).reset_index()\
.set_index(accepted2.loc[accepted2['emp_length'].isnull(), 'emp_length'].index)[0]

accepted2.fillna({'emp_length':pd.cut(x, bins = [0]+list(np.cumsum(a)), labels = a.index)}, inplace = True)

In [19]:
null_sum = accepted2[apriori_columns].isnull().sum()
p_null_sum = null_sum[null_sum > 0]
p_null_sum.sort_values(ascending = False)

num_tl_120dpd_2m              78158
pct_tl_nvr_dlq                70195
total_il_high_credit_limit    70042
num_il_tl                     70042
tot_cur_bal                   70042
total_rev_hi_lim              70042
avg_cur_bal                   70042
mo_sin_old_rev_tl_op          70042
mo_sin_rcnt_rev_tl_op         70042
mo_sin_rcnt_tl                70042
num_accts_ever_120_pd         70042
num_actv_bc_tl                70042
num_actv_rev_tl               70042
num_bc_tl                     70042
tot_coll_amt                  70042
num_op_rev_tl                 70042
num_tl_30dpd                  70042
tot_hi_cred_lim               70042
num_tl_op_past_12m            70042
num_tl_90g_dpd_24m            70042
num_rev_tl_bal_gt_0           70042
num_rev_accts                 70042
num_sats                      58365
num_bc_sats                   58365
bc_util                       53805
percent_bc_gt_75              53711
bc_open_to_buy                53596
mths_since_recent_bc        

### Fill the rest of apriori columns NA's by the median of the sub_grade

In [20]:
null_sum = accepted2[apriori_columns].isnull().sum()
for col in null_sum[null_sum > 0].index:
    a = accepted2.groupby(['sub_grade'])[col].agg('median')
    accepted2.fillna({col:accepted2.loc[accepted2[col].isnull(), 
                                       ['sub_grade', col]].apply(lambda s: a[s[0]], axis = 1)},
                     inplace = True)

In [22]:
null_sum = accepted2[apriori_columns].isnull().sum()
p_null_sum = null_sum[null_sum > 0]
p_null_sum.sort_values(ascending = False)

Series([], dtype: int64)

### removing all the non apriori columns with NA's, only keeping last_credit_pull_d

In [23]:
accepted2.isnull().sum()[accepted2.isnull().sum() > 0]

last_credit_pull_d              38
mo_sin_old_il_acct           83339
mths_since_recent_bc_dlq    366962
mths_since_recent_inq        91827
dtype: int64

In [24]:
accepted2.fillna({'last_credit_pull_d':np.mean(accepted2['last_credit_pull_d'])}, inplace = True)

In [25]:
columns_drop = accepted2.isnull().sum()[accepted2.isnull().sum() > 0].index
accepted2.drop(columns = columns_drop, inplace = True)

### reseting index

In [26]:
accepted2.reset_index(drop = True, inplace = True)

# Feature engeniering

calculate the loan duration in days and months

In [27]:
def loan_length(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

loan_months = []
for end, start in zip(accepted2['last_pymnt_d'], accepted2['issue_d']):
    loan_months.append(loan_length(end, start))


accepted2['duration_days']   = accepted2['last_pymnt_d'] - accepted2['issue_d']
accepted2['duration_months'] = loan_months

In [38]:
accepted2['duration_days'] = accepted2['duration_days'].dt.days

Simplifying loan_status and removing current ones

In [28]:
paid_f = ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']
 
current = ['Current','In Grace Period']

defaulted = ['Default','Does not meet the credit policy. Status:Charged Off', 
             'Charged Off', 'Late (31-120 days)',  'Late (16-30 days)']

accepted2.drop(
    index = accepted2.loc[[True if row in current else False for row in accepted2['loan_status']], 'loan_status'].index,
    inplace = True)

accepted2.loc[:, 'loan_status'] = accepted2['loan_status'].apply(lambda s: 'FullyPaid' if s in paid_f else 'Defaulted')

Calculating the investment return

In [29]:
accepted2['invest_return_per'] = list(map(lambda x: round(x,2), 
                                          (accepted2['total_pymnt'] - accepted2['funded_amnt'])/accepted2['funded_amnt']*100))
accepted2['invest_return'] = list(map(lambda x: round(x,2), accepted2['total_pymnt'] - accepted2['funded_amnt']))

# Saving the new dataset

In [39]:
# accepted2.to_csv('/Users/ivanpassoni/Google Drive/LendingClubData/accepted_b_2015.csv', index = False)

In [34]:
# # To read accepted2
# parse_dates = ['issue_d', 'last_pymnt_d', 'earliest_cr_line', 'last_credit_pull_d']

# accepted2 = pd.read_csv('/Users/ivanpassoni/Google Drive/LendingClubData/accepted_b_2015.csv',
#                        low_memory = False, dtype = dtypes, parse_dates = parse_dates)